# Elo Merchant Category Recommendation - Outlier detection, ensembling
End date: _2019. february 19._<br/>

This tutorial notebook is part of a series for [Elo Mechant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation) contest organized by Elo, one of the largest payment brands in Brazil. It has built partnerships with merchants in order to offer promotions or discounts to cardholders. The objective of the competition is to identify and serve the most relevant opportunities to individuals, by uncovering signals in customer loyalty. LynxKite does not yet support some of the data preprocessing, thus they need to be done in Python. The input files are available from the [download](https://www.kaggle.com/c/elo-merchant-category-recommendation/data) section of the contest:

- **train.csv**,  **test.csv**: list of `card_ids` that can be used for training and testing
- **historical_transactions.csv**: contains up to 3 months' worth of transactions for every card at any of the provided `merchant_ids`
- **new_merchant_transactions.csv**: contains the transactions at new merchants (`merchant_ids` that this particular `card_id` 
has not yet visited) over a period of two months
- **merchants.csv**: contains aggregate information for each `merchant_id` represented in the data set

In [1]:
import gc
import time
import warnings
import datetime
import calendar
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from datetime import date
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
import matplotlib.gridspec as gridspec
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)
gc.enable()

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Input data preparation
### Transactions

In [3]:
df_new_trans = pd.read_csv("preprocessed/trans_merch_new_agg.csv")
df_new_trans = reduce_mem_usage(df_new_trans)

df_hist_trans = pd.read_csv("preprocessed/trans_merch_hist_agg.csv")
df_hist_trans = reduce_mem_usage(df_hist_trans)

Starting memory usage: 289.84 MB
Reduced memory usage: 70.52 MB (75.7% reduction)
Starting memory usage: 325.36 MB
Reduced memory usage: 96.86 MB (70.2% reduction)


In [4]:
df_hist_trans.drop(['Unnamed: 0'], inplace=True, axis=1)
df_new_trans.drop(['Unnamed: 0'], inplace=True, axis=1)

### Train and test data

In [5]:
df_train = pd.read_csv("preprocessed/train_parsed_outlier_marked.csv", index_col="card_id")
df_test = pd.read_csv("preprocessed/test_parsed.csv", index_col="card_id")

### LynxKite export

In [6]:
df_lk_train = pd.read_csv("LynxKite_export/LynxKite_outlier_viral_modeling_train.csv", index_col="card_id")
df_lk_test = pd.read_csv("LynxKite_export/LynxKite_outlier_viral_modeling_test.csv", index_col="card_id")

In [7]:
df_lk_train.drop(['new_id', 'outlier', 'target', 'type'], inplace=True, axis=1)
df_lk_test.drop(['new_id', 'outlier', 'target', 'type'], inplace=True, axis=1)

### Merging

In [8]:
df_train = pd.merge(df_train, df_lk_train, on='card_id', how='left')
df_test = pd.merge(df_test, df_lk_test, on='card_id', how='left')

In [9]:
df_train = pd.merge(df_train, df_hist_trans, on='card_id', how='left')
df_test = pd.merge(df_test, df_hist_trans, on='card_id', how='left')

df_train = pd.merge(df_train, df_new_trans, on='card_id', how='left')
df_test = pd.merge(df_test, df_new_trans, on='card_id', how='left')

In [10]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Starting memory usage: 212.97 MB
Reduced memory usage: 127.67 MB (40.1% reduction)
Starting memory usage: 128.51 MB
Reduced memory usage: 81.70 MB (36.4% reduction)


## Outlier selection
https://www.kaggle.com/sz8416/6-ways-for-feature-selection

### Marking the outliers

In [11]:
df_train['outlier'] = np.where(df_train['target']<-30, 1, 0)

In [12]:
print('There are {:,} marked outliers in the training set.'.format(len(df_train[df_train['outlier'] == 1]['outlier'])))

There are 2,207 marked outliers in the training set.


In [13]:
columns2drop = ['card_id', 'target', 'first_active_month', 'viral_outlier_test', 'viral_outlier_train', 'viral_roles',
                'hist_avg_purchases_lag3_sum', 'hist_avg_purchases_lag3_mean', 'hist_avg_purchases_lag6_sum',
                'hist_avg_purchases_lag6_mean', 'hist_avg_purchases_lag12_sum', 'hist_avg_purchases_lag12_mean',
                'viral_outlier_spread_over_iterations', 'hist_transactions_count', 'hist_purchase_year_min',
                'new_transactions_count', 'new_authorized_flag_mean']

features_train = [c for c in df_train.columns if c not in columns2drop]

### Normalization

In [14]:
df_train_clean = df_train.dropna(how='any', axis=0, subset=features_train)[features_train]

In [15]:
def normalize(df):
    for c in df.columns:
        mean = statistics.mean(df[c])
        std = statistics.stdev(df[c])

        df.loc[:, c] = (df[c] - mean)/std
        print('{}: {:.4f} ({:.4f})'.format(c, mean, std))

    return df

In [16]:
y = df_train_clean['outlier']
print('There are {:,} records in the outlier set.'.format(len(y)))

df_train_clean.drop(['outlier'], axis=1, inplace=True)
X = normalize(df_train_clean)
print('There are {:,} features and {:,} items in the training set.'.format(X.shape[1], X.shape[0]))

There are 146,616 records in the outlier set.
feature_1: 3.0871 (1.1958)
feature_2: 1.7319 (0.7497)
feature_3: 0.5536 (0.4971)
elapsed_days: 361.0683 (285.5164)
year: 2016.5552 (0.7572)
month: 7.5153 (3.3234)
days_feature1: 1152.3708 (1082.4652)
days_feature2: 657.3871 (707.0489)
days_feature3: 226.1918 (318.1874)
number_of_transactions_x: 100.9815 (113.0187)
merch_seg_viral_outlier_average_after_iteration_1_most_common: 0.0028 (0.0324)
merch_seg_viral_outlier_average_after_iteration_2_most_common: 0.0018 (0.0244)
merch_seg_viral_outlier_average_after_iteration_3_most_common: 0.0018 (0.0244)
merch_seg_viral_outlier_average_after_iteration_4_most_common: 0.0018 (0.0244)
merch_seg_viral_outlier_average_after_iteration_5_most_common: 0.0018 (0.0244)
merch_seg_viral_outlier_standard_deviation_after_iteration_1_most_common: 0.0102 (0.0456)
merch_seg_viral_outlier_standard_deviation_after_iteration_2_most_common: 0.0075 (0.0353)
merch_seg_viral_outlier_standard_deviation_after_iteration_3_mo

new_category_2_merch_mean: 2.2377 (1.4758)
new_category_3_sum: 4.5775 (6.9506)
new_category_3_mean: 0.6156 (0.6155)
new_category_4_sum: 3.9364 (5.1460)
new_category_4_mean: 0.4770 (0.3884)
new_city_id_trans_nunique: 2.5839 (1.7275)
new_city_id_merch_nunique: 2.3113 (1.3520)
new_installments_sum: 5.3500 (8.7879)
new_installments_median: 0.5993 (0.7816)
new_installments_mean: 0.6973 (0.9062)
new_installments_max: 1.5637 (3.5770)
new_installments_min: 0.2152 (0.7373)
new_installments_std: 0.5075 (1.0764)
new_merchant_id_nunique: 7.8677 (6.7754)
new_merchant_category_id_trans_nunique: 6.2233 (4.1856)
new_merchant_category_id_merch_nunique: 6.1393 (4.1487)
new_merchant_group_id_nunique: 6.2793 (5.2163)
new_month_lag_max: 1.8792 (0.3258)
new_month_lag_min: 1.0982 (0.2976)
new_month_lag_mean: 1.4756 (0.2884)
new_month_lag_var: 0.2096 (0.1335)
new_most_recent_sales_range_sum: 15.9722 (13.9074)
new_most_recent_sales_range_mean: 2.0686 (0.6674)
new_most_recent_sales_range_max: 3.4746 (0.8204)
ne

In [17]:
X.shape, y.shape

((146616, 274), (146616,))

In [18]:
X.isnull().sum()

feature_1                                                                   0
feature_2                                                                   0
feature_3                                                                   0
elapsed_days                                                                0
year                                                                        0
month                                                                       0
days_feature1                                                               0
days_feature2                                                               0
days_feature3                                                               0
number_of_transactions_x                                                    0
merch_seg_viral_outlier_average_after_iteration_1_most_common               0
merch_seg_viral_outlier_average_after_iteration_2_most_common               0
merch_seg_viral_outlier_average_after_iteration_3_most_common   

### Feature selection

#### Pearson correlation

In [19]:
def cor_selector(X, y, limit=100):
    cor_list = []
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)

    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-limit:]].columns.tolist()
    cor_support = [True if i in cor_feature else False for i in X.columns.tolist()]
    return cor_support, cor_feature, cor_list

In [20]:
cor_support, cor_feature, cor_value = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

100 selected features


#### Chi-2

In [21]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest

X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=100)
chi_selector.fit(X_norm, y)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


SelectKBest(k=100, score_func=<function chi2 at 0x7f94891f59d8>)

In [22]:
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

100 selected features


#### RFE

In [23]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=5, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 274 features.
Fitting estimator with 264 features.
Fitting estimator with 254 features.
Fitting estimator with 244 features.
Fitting estimator with 234 features.
Fitting estimator with 224 features.
Fitting estimator with 214 features.
Fitting estimator with 204 features.
Fitting estimator with 194 features.
Fitting estimator with 184 features.
Fitting estimator with 174 features.
Fitting estimator with 164 features.
Fitting estimator with 154 features.
Fitting estimator with 144 features.
Fitting estimator with 134 features.
Fitting estimator with 124 features.
Fitting estimator with 114 features.
Fitting estimator with 104 features.
Fitting estimator with 94 features.
Fitting estimator with 84 features.
Fitting estimator with 74 features.
Fitting estimator with 64 features.
Fitting estimator with 54 features.
Fitting estimator with 44 features.
Fitting estimator with 34 features.
Fitting estimator with 24 features.
Fitting estimator with 14 features.


RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
  n_features_to_select=5, step=10, verbose=5)

In [24]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

5 selected features


#### Embedded

In [25]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), '1.25*median')
embeded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False,
        threshold='1.25*median')

In [26]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

274 selected features


#### Random Forest

In [27]:
embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=5), threshold='1.25*median')
embeded_rf_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        max_features=None, norm_order=1, prefit=False,
        threshold='1.25*median')

In [28]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

112 selected features


#### LightGBM

In [29]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
embeded_lgb_selector.fit(X, y)

SelectFromModel(estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.2,
        importance_type='split', learning_rate=0.05, max_depth=-1,
        min_child_samples=20, min_child_weight=40, min_split_gain=0.01,
        n_estimators=500, n_jobs=-1, num_leaves=32, objective=None,
        random_state=None, reg_alpha=3, reg_lambda=1, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
        max_features=None, norm_order=1, prefit=False,
        threshold='1.25*median')

In [30]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

109 selected features


In [31]:
pd.set_option('display.max_rows', None)

feature_selection_df = pd.DataFrame({
    'feature': X.columns.tolist(),
    'Pearson': cor_support,
    'Chi-2': chi_support,
    'RFE': rfe_support,
    'Logistics': embeded_lr_support,
    'Random Forest': embeded_rf_support,
    'LightGBM': embeded_lgb_support
})

feature_selection_df['total'] = np.sum(feature_selection_df, axis=1)

feature_selection_df = feature_selection_df.sort_values(['total', 'feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df[:50]

,feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,total
1,viral_outlier_after_iteration_5,True,True,True,True,True,True,6
2,viral_outlier_after_iteration_4,True,True,True,True,True,True,6
3,new_purchase_weekofyear_nunique,True,True,True,True,True,True,6
4,viral_outlier_after_iteration_3,True,True,False,True,True,True,5
5,viral_outlier_after_iteration_2,True,True,False,True,True,True,5
6,viral_outlier_after_iteration_1,True,True,False,True,True,True,5
7,new_purchase_weekofyear_mean,True,True,False,True,True,True,5
8,new_purchase_weekofyear_max,True,True,False,True,True,True,5
9,new_purchase_quarter_max,True,True,False,True,True,True,5
10,new_purchase_month_max,True,True,False,True,True,True,5


In [32]:
len(feature_selection_df[feature_selection_df['total'] > 5]['feature'])

3

In [33]:
list(feature_selection_df[feature_selection_df['total'] > 5]['feature'])

['viral_outlier_after_iteration_5',
 'viral_outlier_after_iteration_4',
 'new_purchase_weekofyear_nunique']

### Training

In [ ]:
# all features
features = [c for c in df_train.columns if c not in columns2drop]

# features used for training the outlier detector
#features = ['viral_outlier_after_iteration_5', 'viral_outlier_after_iteration_4']

df_train_clean = df_train.dropna(how='any', axis=0, subset=features)[features]
y = df_train_clean.outlier

df_train_clean.drop(['outlier'], axis=1, inplace=True)
X = normalize(df_train_clean)

features_test = features
features_test.remove('outlier')
X_test = df_test.dropna(how='any', axis=0, subset=features_test)[features_test]

In [ ]:
X.shape, y.shape, X_test.shape

#### Random Forest

In [ ]:
clf = RandomForestClassifier(verbose=1)
clf.fit(X, y)

In [ ]:
randomforest_outlier_pred = clf.predict(X_test)
randomforest_outlier_pred_proba = clf.predict_proba(X_test)

In [ ]:
randomforest_outlier_pred.sum()

In [ ]:
randomforest_outlier_pred_proba.sum()

In [ ]:
rf_outlier_card_ids = []
for i in range(len(randomforest_outlier_pred)):
    if randomforest_outlier_pred[i] == 1:
        print('{:,}. card_id: {}'.format(i, df_test_clean['card_id'].iloc[i]))
        rf_outlier_card_ids.append(df_test_clean['card_id'].iloc[i])

In [ ]:
rf_outlier_card_ids

#### Logistic regression

In [ ]:
clf = LogisticRegression(verbose=1)
clf.fit(X, y)

In [ ]:
logistic_regression_outlier_pred = clf.predict(X_test)

In [ ]:
logistic_regression_outlier_pred.sum()

In [ ]:
lr_outlier_card_ids = []
for i in range(len(logistic_regression_outlier_pred)):
    if logistic_regression_outlier_pred[i] == 1:
        print('{:,}. card_id: {}'.format(i, df_test_clean['card_id'].iloc[i]))
        lr_outlier_card_ids.append(df_test_clean['card_id'].iloc[i])

#### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
clf.fit(X, y)

In [ ]:
adaboost_outlier_pred = clf.predict(X_test)

In [ ]:
adaboost_outlier_pred.sum()

In [ ]:
adaboost_outlier_card_ids = []
for i in range(len(adaboost_outlier_pred)):
    if adaboost_outlier_pred[i] == 1:
        print('{:,}. card_id: {}'.format(i, df_test_clean['card_id'].iloc[i]))
        adaboost_outlier_card_ids.append(df_test_clean['card_id'].iloc[i])

#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

clf = GradientBoostingRegressor({
    'n_estimators': 500,
    'max_depth': 4,
    'min_samples_split': 2,
    'learning_rate': 0.01,
    'loss': 'ls'
})

clf.fit(X, y)

In [ ]:
gradient_boosting_outlier_pred = clf.predict(X_test)

### Ensembling

In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [ ]:
print('Size of the output of the logistic regression: {}\nSize of the output of the random forest: {}\nSize of the output of the random forest: {}'.format(len(lr_outlier_card_ids), len(rf_outlier_card_ids), len(adaboost_outlier_card_ids)))

In [ ]:
intersection(lr_outlier_card_ids, rf_outlier_card_ids)

In [ ]:
intersection(lr_outlier_card_ids, adaboost_outlier_card_ids)

In [ ]:
intersection(rf_outlier_card_ids, adaboost_outlier_card_ids)

## Submission

In [ ]:
clf = lgb.Booster(model_file='models/lightgbm_all.txt')

In [ ]:
drops = ['card_id', 'first_active_month', 'target', 'outlier']
use_cols = [c for c in df_train.columns if c not in drops]
features = list(df_train[use_cols].columns)

In [ ]:
predictions = clf.predict(df_test[features])

In [ ]:
df_sub = pd.DataFrame({
    "card_id": df_test["card_id"].values
})
df_sub["target"] = predictions

In [ ]:
# Updating the intersection of RF & AdaBoost
df_sub.loc[df_sub['card_id'] == 'C_ID_aae50409e7', 'target'] = -33.218750

In [ ]:
for i in range(len(lr_outlier_card_ids)):
    print('The value of {} is {}'.format(lr_outlier_card_ids[i], df_sub.loc[df_sub['card_id'] == lr_outlier_card_ids[i], 'target'].values[0]))
    df_sub.loc[df_sub['card_id'] == lr_outlier_card_ids[i], 'target'] = -33.218750

In [ ]:
len(df_sub[df_sub['target'] < -30])

In [ ]:
df_sub.to_csv("output/lgbm_rf_and_adaboost_outliers.csv", index=False)

* Random Forest (LB score: 5.994)
* Logistic Regression (LB score: 5.990)
* Random Forest & AdaBoost (LB score: 5.986)